This is to scrap lyrics from the website.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

Very old method: get the element and extract the relevant text. Because the website dynamic action making it hard to extract all links.

In [ ]:
from bs4 import BeautifulSoup

# Open and read the file
with open('element_all_song.txt', 'r') as f:
    html = f.read()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Get the divs for the song titles and the a tags for the song URLs
titles = soup.select('div.mini_card-title')
urls = soup.select('mini-song-card a')

# Extract the URLs from each a tag
urls = [url['href'] for url in urls]

# Extract the Chinese and English title from each title div
chinese_titles = []
english_titles = []

for title in titles:
    parts = title.text.split(' (')
    chinese_titles.append(parts[0])
    if len(parts) > 1:
        english_titles.append(parts[1].replace(')', ''))
    else:
        english_titles.append(None)

# Print the URLs and titles
for url, chi_title, eng_title in zip(urls, chinese_titles, english_titles):
    print(f"URL: {url}")
    print(f"Chinese Title: {chi_title}")
    print(f"English Title: {eng_title}")
    print()


Make it a data frame.

In [ ]:
df = pd.DataFrame({
    'URL': urls,
    'Title': chinese_titles,
    'English Title': english_titles
})

print(df)

For each song page, extract the lyrics

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm  # This library is for progress bars. It's optional.

# Initialize an empty list to store the lyrics.
lyrics_list = []

# Loop over each URL in the DataFrame.
for url in tqdm(df['URL']):
    # Send a GET request to the URL.
    response = requests.get(url)
    
    # Parse the HTML content of the page with BeautifulSoup.
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the lyrics div and extract the text.
    lyrics_div = soup.find('div', attrs={"data-lyrics-container": "true"})
    lyrics = lyrics_div.get_text(separator='\n')
    
    # Append the lyrics to the lyrics_list.
    lyrics_list.append(lyrics)

# Add the lyrics_list as a new column in the DataFrame.
df['Lyrics'] = lyrics_list


Convert it to JSON file to match the template.

In [ ]:
import pandas as pd
import jieba

# Assuming df is your DataFrame
# df = pd.DataFrame({
#     'URL': urls,
#     'Title': chinese_titles,
#     'English Title': english_titles,
#     'Lyrics': lyrics_list
# })

def get_chunks(content, url, title):
    content_length = len(content)
    content_tokens = len(list(jieba.cut(content, cut_all=False)))
    chunk = {
        "essay_title": title,
        "essay_url": url,
        "essay_date": "NA",
        "essay_thanks": "NA",
        "content": content,
        "content_length": content_length,
        "content_tokens": content_tokens,
        "embedding": []
    }
    return [chunk]  # As chunks is an array, wrap the chunk object in a list

data = {
    "current_date": "2023-03-01",
    "author": "Your name here",
    "url": "Your URL here",
    "length": df['Lyrics'].apply(len).sum().item(),
    "tokens": df['Lyrics'].apply(lambda x: len(list(jieba.cut(x, cut_all=False)))).sum().item(),
    "essays": []
}

for idx, row in df.iterrows():
    essay = {
        "title": row['Title'],
        "url": row['URL'],
        "date": "NA",
        "thanks": "NA",
        "content": row['Lyrics'],
        "length": len(row['Lyrics']),
        "tokens": len(list(jieba.cut(row['Lyrics'], cut_all=False))),
        "chunks": get_chunks(row['Lyrics'], row['URL'], row['Title'])
    }
    data["essays"].append(essay)

# At this point, `data` is a Python dictionary that matches the PGJSON structure.
# You can convert it to a JSON string using json.dumps:
import json
json_str = json.dumps(data, ensure_ascii=False)

# Write the JSON string to a file named pg.json
with open('pg.json', 'w', encoding='utf8') as f:
    f.write(json_str)
